# Equation of state (EOS)
For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Example: Silicon
### Prepare the inputs and visualize the WorkGraph

In [1]:
from aiida import load_profile, orm
from ase.build import bulk
from workgraph_collections.qe.eos import EosWorkgraph

load_profile()

structure = orm.StructureData(ase=bulk("Si"))
code = orm.load_code("pw-7.4@localhost")
parameters = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "smearing",
        "smearing": "gaussian",
        "degauss": 0.1,
    },
}
# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}

# ===============================================================================
wg = EosWorkgraph.build_graph(
    structure=structure,
    code=code,
    scales=[0.98, 0.99, 1.0, 1.01, 1.02],
    parameters=parameters,
    kpoints=kpoints,
    pseudos=pseudos,
    metadata=metadata,
)
wg.to_html()

### Run the workflow

In [2]:
wg.run()

09/02/2025 01:10:17 PM <1293550> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175182|WorkGraphEngine|continue_workgraph]: tasks ready to run: scale_structure
09/02/2025 01:10:17 PM <1293550> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175182|WorkGraphEngine|update_task_state]: Task: scale_structure, type: CALCFUNCTION, finished.
09/02/2025 01:10:17 PM <1293550> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175182|WorkGraphEngine|continue_workgraph]: tasks ready to run: all_scf
09/02/2025 01:10:19 PM <1293550> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175182|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 175190
09/02/2025 01:10:21 PM <1293550> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175190|WorkGraphEngine|continue_workgraph]: tasks ready to run: PwCalculation,PwCalculation1,PwCalculation2,PwCalculation3,PwCalculation4
09/02/2025 01:10:23 PM <

## Print the results

In [3]:
#------------------------- Print the output -------------------------
data = wg.outputs.result.value.get_dict()
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))



Result: 
B: 83.930472220061
v0: 40.947349586558
e0: -308.18973212512
v0: 40.947349586558
